# 📥 Terabox Downloader for Google Colab

This notebook allows you to download files from Terabox share links directly in Google Colab.

**Features:**
- Download files from Terabox share URLs
- Progress tracking
- Batch downloads
- Files saved to `/content/downloads/`


## 🔧 Setup
Run this cell first to install required packages and set up the environment.

In [ ]:
# Install required packages
!pip install requests pillow

# Create downloads directory
import os
os.makedirs('/content/downloads', exist_ok=True)

print("✅ Setup complete!")

## 📚 Core Classes
This cell contains the main downloader classes.

In [ ]:
import requests
import json
import re
import os
import time
from urllib.parse import urlparse, parse_qs
from datetime import datetime

class TeraboxAPI:
    def __init__(self):
        self.session = requests.Session()
        self.session.headers.update({
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        })
        self.primary_api_url = "https://ashlynn.serv00.net/Ashlynnterabox.php"
        
    def get_file_info(self, terabox_url):
        try:
            params = {'url': terabox_url}
            response = self.session.get(self.primary_api_url, params=params, timeout=30)
            response.raise_for_status()
            
            if response.headers.get('content-type', '').startswith('application/json'):
                data = response.json()
            else:
                try:
                    data = json.loads(response.text)
                except:
                    return self._parse_html_response(response.text, terabox_url)
                    
            return self._parse_json_response(data, terabox_url)
            
        except Exception as e:
            print(f"API Error: {e}")
            return None
            
    def _parse_json_response(self, data, original_url):
        try:
            if isinstance(data, dict):
                download_url = None
                filename = None
                file_size = None
                
                # Try different field names for download URL
                for field in ['download_url', 'url', 'direct_url', 'link', 'dlink']:
                    if field in data and data[field]:
                        download_url = data[field]
                        break
                        
                # Try different field names for filename
                for field in ['filename', 'name', 'title', 'server_filename']:
                    if field in data and data[field]:
                        filename = data[field]
                        break
                        
                # Try different field names for size
                for field in ['size', 'file_size', 'filesize']:
                    if field in data and data[field]:
                        file_size = data[field]
                        break
                        
                if download_url:
                    return {
                        'download_url': download_url,
                        'filename': filename or self._extract_filename_from_url(download_url),
                        'size': file_size,
                        'size_formatted': self._format_file_size(file_size) if file_size else 'Unknown',
                        'original_url': original_url
                    }
                    
        except Exception as e:
            print(f"JSON parsing error: {e}")
            
        return None
        
    def _parse_html_response(self, html_content, original_url):
        try:
            patterns = [
                r'href="([^"]*terabox[^"]*")',
                r'"download_url":"([^"]*")',
                r'"url":"([^"]*")'
            ]
            
            for pattern in patterns:
                matches = re.findall(pattern, html_content, re.IGNORECASE)
                if matches:
                    download_url = matches[0]
                    if self._is_valid_download_url(download_url):
                        return {
                            'download_url': download_url,
                            'filename': self._extract_filename_from_url(download_url),
                            'size': None,
                            'size_formatted': 'Unknown',
                            'original_url': original_url
                        }
        except Exception as e:
            print(f"HTML parsing error: {e}")
            
        return None
        
    def _is_valid_download_url(self, url):
        try:
            parsed = urlparse(url)
            return parsed.scheme in ['http', 'https'] and parsed.netloc
        except:
            return False
            
    def _extract_filename_from_url(self, url):
        try:
            parsed = urlparse(url)
            filename = parsed.path.split('/')[-1] if '/' in parsed.path else parsed.path
            return filename if filename and '.' in filename else f"terabox_file_{int(time.time())}.bin"
        except:
            return f"terabox_file_{int(time.time())}.bin"
            
    def _format_file_size(self, size):
        try:
            if isinstance(size, str):
                numbers = re.findall(r'\d+\.?\d*', size)
                if numbers:
                    size = float(numbers[0])
                    if 'GB' in size.upper():
                        size *= 1024 * 1024 * 1024
                    elif 'MB' in size.upper():
                        size *= 1024 * 1024
                    elif 'KB' in size.upper():
                        size *= 1024
                        
            if isinstance(size, (int, float)):
                if size >= 1024 * 1024 * 1024:
                    return f"{size / (1024 * 1024 * 1024):.2f} GB"
                elif size >= 1024 * 1024:
                    return f"{size / (1024 * 1024):.2f} MB"
                elif size >= 1024:
                    return f"{size / 1024:.2f} KB"
                else:
                    return f"{size} B"
        except:
            pass
            
        return str(size) if size else 'Unknown'

class URLValidator:
    def __init__(self):
        self.terabox_domains = [
            'terabox.com', 'www.terabox.com', '1024terabox.com',
            'teraboxapp.com', 'nephobox.com', 'dubox.com', '4funbox.com'
        ]
        
    def is_valid_terabox_url(self, url):
        try:
            parsed = urlparse(url.lower())
            return any(domain in parsed.netloc for domain in self.terabox_domains) and ('/s/' in parsed.path or 'surl=' in parsed.query)
        except:
            return False

class ColabTeraboxDownloader:
    def __init__(self):
        self.api = TeraboxAPI()
        self.validator = URLValidator()
        
    def download_file(self, terabox_url, download_path="/content/downloads"):
        print(f"🔗 Processing URL: {terabox_url}")
        
        if not self.validator.is_valid_terabox_url(terabox_url):
            print("❌ Invalid Terabox URL")
            return False, "Invalid Terabox URL"
        
        print("📡 Getting file information...")
        file_info = self.api.get_file_info(terabox_url)
        
        if not file_info:
            print("❌ Could not extract download link")
            return False, "Could not extract download link"
        
        os.makedirs(download_path, exist_ok=True)
        
        download_url = file_info['download_url']
        filename = file_info.get('filename', 'downloaded_file')
        filepath = os.path.join(download_path, filename)
        
        print(f"📁 Downloading: {filename}")
        print(f"📊 Size: {file_info.get('size_formatted', 'Unknown')}")
        
        try:
            response = requests.get(download_url, stream=True)
            response.raise_for_status()
            
            total_size = int(response.headers.get('content-length', 0))
            downloaded = 0
            
            with open(filepath, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    if chunk:
                        f.write(chunk)
                        downloaded += len(chunk)
                        
                        if total_size > 0:
                            progress = (downloaded / total_size) * 100
                            print(f"\r⏳ Progress: {progress:.1f}%", end="", flush=True)
            
            print(f"\n✅ Download complete: {filepath}")
            print(f"📂 File saved to: {filepath}")
            return True, filepath
            
        except Exception as e:
            print(f"\n❌ Download failed: {str(e)}")
            return False, f"Download failed: {str(e)}"
    
    def download_multiple(self, urls):
        print(f"📥 Starting batch download of {len(urls)} files...")
        results = []
        
        for i, url in enumerate(urls, 1):
            print(f"\n[{i}/{len(urls)}] Processing...")
            success, result = self.download_file(url)
            results.append((url, success, result))
            
            if i < len(urls):
                time.sleep(1)  # Small delay between downloads
        
        # Summary
        successful = sum(1 for _, success, _ in results if success)
        print(f"\n📋 Batch download complete: {successful}/{len(urls)} successful")
        
        return results

# Create downloader instance
downloader = ColabTeraboxDownloader()
print("🚀 Terabox Downloader ready!")

## 🚀 Usage Examples

### Single File Download
Replace the URL with your Terabox share link:

In [ ]:
# Single file download
terabox_url = "https://terabox.com/s/your-share-link-here"

success, result = downloader.download_file(terabox_url)

if success:
    print(f"File downloaded to: {result}")
else:
    print(f"Download failed: {result}")

### Batch Download
Download multiple files at once:

In [ ]:
# Batch download
urls = [
    "https://terabox.com/s/link1",
    "https://terabox.com/s/link2",
    "https://terabox.com/s/link3"
]

results = downloader.download_multiple(urls)

# Show detailed results
for url, success, result in results:
    status = "✅" if success else "❌"
    print(f"{status} {url}: {result}")

## 📂 View Downloaded Files
Check what files have been downloaded:

In [ ]:
# List downloaded files
import os
from IPython.display import display, HTML

download_dir = "/content/downloads"

if os.path.exists(download_dir):
    files = os.listdir(download_dir)
    if files:
        print(f"📁 Downloaded files ({len(files)} total):")
        for file in files:
            filepath = os.path.join(download_dir, file)
            size = os.path.getsize(filepath)
            size_mb = size / (1024 * 1024)
            print(f"  📄 {file} ({size_mb:.2f} MB)")
    else:
        print("📭 No files downloaded yet")
else:
    print("📂 Downloads directory not found")

# Show disk usage
!du -sh /content/downloads 2>/dev/null || echo "Downloads folder is empty"

## 💾 Download Files to Your Computer
Download the files from Colab to your local computer:

In [ ]:
# Create a zip file of all downloads
import zipfile
from google.colab import files

download_dir = "/content/downloads"
zip_path = "/content/terabox_downloads.zip"

if os.path.exists(download_dir) and os.listdir(download_dir):
    with zipfile.ZipFile(zip_path, 'w') as zipf:
        for file in os.listdir(download_dir):
            file_path = os.path.join(download_dir, file)
            zipf.write(file_path, file)
    
    print(f"📦 Created zip file: {zip_path}")
    print("⬇️ Downloading to your computer...")
    files.download(zip_path)
else:
    print("❌ No files to download")

## 📝 Notes

- **File Location**: All downloaded files are saved to `/content/downloads/`
- **File Persistence**: Files will be deleted when the Colab session ends
- **Download Limit**: Colab has storage limits (usually ~78GB)
- **API Reliability**: Uses free Terabox API services which may have limitations
- **Network Speed**: Download speed depends on Colab's internet connection

### Troubleshooting
- If downloads fail, try running the setup cell again
- Ensure your Terabox URLs are valid share links
- Check that the files aren't too large for Colab's storage
